In [19]:
!mkdir -p bbmodcache
!curl http://bb-ai.net.s3.amazonaws.com/bb-python-modules/bbSearch.py > bbmodcache/bbSearch.py
from bbmodcache.bbSearch import SearchProblem, search

A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file bbmodcache already exists.
Error occurred while processing: bbmodcache.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 18767  100 18767    0     0  36774      0 --:--:-- --:--:-- --:--:-- 36870


In [20]:
class Robot:  #meds_on_hand = meds_on_hand rooms
    def __init__(self, location, meds_on_hand, battery):
        self.location      = location
        self.meds_on_hand = meds_on_hand
        self.battery      = battery
        
    def weight_carried(self):
        return sum([ITEM_WEIGHT[i] for i in self.meds_on_hand])
    
    ## Define unique string representation for the state of the robot object
    def __repr__(self):
        return str( ( self.location, 
                      self.meds_on_hand,
                      self.battery ) )
            
class Door:
    def __init__(self, roomA, roomB, doorkey=None, locked=False):
        self.goes_between = {roomA, roomB}
        self.doorkey      = doorkey
        self.locked       = locked
        # Define handy dictionary to get room on other side of a door
        self.other_loc = {roomA:roomB, roomB:roomA}
        
    ## Define a unique string representation for a door object    
    def __repr__(self):
        return str( ("door", self.goes_between, self.doorkey, self.locked) )

In [21]:
class State:
    def __init__( self, robot, doors, room_contents ):
        self.robot = robot
        self.doors = doors
        self.room_contents = room_contents
        
    ## Define a string representation that will be uniquely identify the state.
    ## An easy way is to form a tuple of representations of the components of 
    ## the state, then form a string from that:
    def __repr__(self):
        return str( ( self.robot.__repr__(),
                      [d.__repr__() for d in self.doors],
                      self.room_contents ) )
  

In [22]:
ROOM_CONTENTS = {
    'Doctor Office'     : {'AOTD','WasteE'},
    'Patient Ward'   : {'WasteA', 'WasteB'},
    'Pharmacy' : {'PresPackageA', 'PresPackageB', 'PresPackageC', 'PresPackageD','WasteD'},
    'Waste Disposal' : {'WasteC'},
}

ITEM_WEIGHT = {
          'AOTD' : 5,
        'WasteA' : 23,
        'WasteB' : 23,
  'PresPackageD' : 15,
  'PresPackageA' : 15,
  'PresPackageB' : 15,
  'PresPackageC' : 15,
        'WasteC' : 23,
    'WasteD' : 20,
    'WasteE' : 20,
}
          
DOORS = [
    Door('Doctor Office', 'Patient Ward' ),
    Door( 'Patient Ward', 'Pharmacy', doorkey='AOTD', locked=False ), #Pharmacy, PatientWard, Waste
    Door('Patient Ward','Waste Disposal')
]    
    

In [23]:
from copy import deepcopy

class RobotWorker( SearchProblem ):
    
    def __init__( self, state, goal_item_locations ):
        self.initial_state = state
        self.goal_item_locations = goal_item_locations
        
    def possible_actions( self, state ):
        
        robot_location = state.robot.location
        battery       = state.robot.battery
        weight_carried = state.robot.weight_carried()
        
        actions = []
        # Can put down any meds_on_hand
        for i in state.robot.meds_on_hand:
            actions.append( ("put down", i) )

        # Can pick up any item in room if strong enough    
        for i in state.room_contents[robot_location]:
            if battery >= weight_carried + ITEM_WEIGHT[i]:
                actions.append( ("pick up", i))
                
        # If there is an unlocked door between robot location and
        # another location can move to that location
        for door in state.doors:
            if  door.locked==False and robot_location in door.goes_between:
                actions.append( ("move to", door.other_loc[robot_location]) )
                
        # Now the actions list should contain all possible actions
        return actions
    
    def successor( self, state, action):
        next_state = deepcopy(state)
        act, target = action
        if act== "put down":
            next_state.robot.meds_on_hand.remove(target)
            next_state.room_contents[state.robot.location].add(target)
            
        if act == "pick up":
            next_state.robot.meds_on_hand.append(target)
            next_state.room_contents[state.robot.location].remove(target)
            
        if act == "move to":
            next_state.robot.location = target
            
        return next_state
        
    def goal_test(self, state):
        #print(state.room_contents)
        for room, contents in self.goal_item_locations.items():
            for i in contents:
                if not i in state.room_contents[room]:
                    return False
        return True
    
    def display_state(self,state):
        print("Robot location:", state.robot.location)
        print("Robot carrying:", state.robot.meds_on_hand)
        print("Room contents:", state.room_contents)

In [24]:
rob = Robot('Waste Disposal', [], 150 )
        
state = State(rob, DOORS, ROOM_CONTENTS)

#goal_item_locations = {"Patient Ward" : {"PresPackageA","PresPackageB"},
 #                      "Waste Disposal" :{"WasteA","WasteB"}} 
#"Patient Ward":{"PresPackageA", "PresPackageB", "PresPackageC", "PresPackageD"},

goal_item_locations = {"Patient Ward":{"PresPackageA", "PresPackageB"},
                       "Waste Disposal" :{"WasteA","WasteB","WasteD","WasteE"}}
        
RW_PROBLEM_1 = RobotWorker( state, goal_item_locations )    

In [25]:
poss_acts = RW_PROBLEM_1.possible_actions( RW_PROBLEM_1.initial_state )
poss_acts

[('pick up', 'WasteC'), ('move to', 'Patient Ward')]

In [26]:
for act in poss_acts:
    print("Action", act, "leads to the following state:")
    next_state = RW_PROBLEM_1.successor( RW_PROBLEM_1.initial_state, act )
    RW_PROBLEM_1.display_state(next_state)
    print()

Action ('pick up', 'WasteC') leads to the following state:
Robot location: Waste Disposal
Robot carrying: ['WasteC']
Room contents: {'Doctor Office': {'WasteE', 'AOTD'}, 'Patient Ward': {'WasteA', 'WasteB'}, 'Pharmacy': {'PresPackageD', 'PresPackageC', 'PresPackageA', 'PresPackageB', 'WasteD'}, 'Waste Disposal': set()}

Action ('move to', 'Patient Ward') leads to the following state:
Robot location: Patient Ward
Robot carrying: []
Room contents: {'Doctor Office': {'WasteE', 'AOTD'}, 'Patient Ward': {'WasteA', 'WasteB'}, 'Pharmacy': {'PresPackageD', 'PresPackageC', 'PresPackageA', 'PresPackageB', 'WasteD'}, 'Waste Disposal': {'WasteC'}}



In [27]:
#TESTS

In [28]:
#Heuristic1


In [29]:
def MoveMeds(state):
    counter = 0
    for G_room,contents in goal_item_locations.items():
        for item in contents:
            if not item in state.room_contents[G_room]:
                counter = counter + 1
    return counter

In [30]:
#Heuristic2


In [31]:
def TasksRemaining(state):
    counter = 0
    rooms = 0
    for G_room, items_to_move in goal_item_locations.items():
        rooms = rooms + 1
        room_complete = True
        for item in items_to_move:
            if not item in state.room_contents[G_room]:
                room_complete = False
                break
        if room_complete:
            counter = counter + 1
    return rooms -counter

In [32]:
search( RW_PROBLEM_1, 'BF/FIFO', 5000, loop_check=False, heuristic=TasksRemaining)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=TasksRemaining
Max search nodes: 5000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

!! Search node limit (5000) reached !!
): No solution found :(


SEARCH SPACE STATS:
Total nodes generated          =     5001  (includes start)
Nodes tested (by goal_test)    =      917  (all expanded)
Nodes left in queue            =     4083

Time taken = 0.6977 seconds



'NODE_LIMIT_EXCEEDED'

In [33]:
search( RW_PROBLEM_1, 'BF/FIFO', 50000, loop_check=False, heuristic=TasksRemaining)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=TasksRemaining
Max search nodes: 50000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
.........
!! Search node limit (50000) reached !!
): No solution found :(


SEARCH SPACE STATS:
Total nodes generated          =    50001  (includes start)
Nodes tested (by goal_test)    =     9013  (all expanded)
Nodes left in queue            =    40987

Time taken = 7.5138 seconds



'NODE_LIMIT_EXCEEDED'

In [35]:
search( RW_PROBLEM_1, 'BF/FIFO', 500000, loop_check=False, heuristic=TasksRemaining)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=TasksRemaining
Max search nodes: 500000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
........................................................................................
!! Search node limit (500000) reached !!
): No solution found :(


SEARCH SPACE STATS:
Total nodes generated          =   500001  (includes start)
Nodes tested (by goal_test)    =    88716  (all expanded)
Nodes left in queue            =   411284

Time taken = 187.9039 seconds



'NODE_LIMIT_EXCEEDED'

In [36]:
search( RW_PROBLEM_1, 'BF/FIFO', 5000000, loop_check=False, heuristic=TasksRemaining)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=TasksRemaining
Max search nodes: 5000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
.................................................................................................... (100000)
.................................................................................................... (200000)
.................................................................................................... (300000)
.................................................................................................... (400000)
.................................................................................................... (500000)
.................................................................................................... (600000)
.....................

'NODE_LIMIT_EXCEEDED'

In [ ]:
def cost(path,state):
    return len(path)

In [ ]:
#A* Case 2


In [ ]:
search( RW_PROBLEM_1, 'BF/FIFO', 5000, loop_check=True, heuristic=TasksRemaining, cost = cost)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=cost, heuristic=TasksRemaining
Max search nodes: 5000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
.
!! Search node limit (5000) reached !!
): No solution found :(


SEARCH SPACE STATS:
Total nodes generated          =     8710  (includes start)
Nodes discarded by loop_check  =     3709  (5001 distinct states added to queue)
Nodes tested (by goal_test)    =     1370  (all expanded)
Nodes left in queue            =     3630

Time taken = 1.0947 seconds



'NODE_LIMIT_EXCEEDED'

In [ ]:
search( RW_PROBLEM_1, 'BF/FIFO', 50000, loop_check=True, heuristic=TasksRemaining, cost = cost)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=cost, heuristic=TasksRemaining
Max search nodes: 50000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
...............
!! Search node limit (50000) reached !!
): No solution found :(


SEARCH SPACE STATS:
Total nodes generated          =   105511  (includes start)
Nodes discarded by loop_check  =    55510  (50001 distinct states added to queue)
Nodes tested (by goal_test)    =    15507  (all expanded)
Nodes left in queue            =    34493

Time taken = 14.4487 seconds



'NODE_LIMIT_EXCEEDED'

In [ ]:
search( RW_PROBLEM_1, 'BF/FIFO', 500000, loop_check=True, heuristic=TasksRemaining, cost = cost)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=cost, heuristic=TasksRemaining
Max search nodes: 500000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
.................................................................................................... (100000)
..............................................................................
!! Search node limit (500000) reached !!
): No solution found :(


SEARCH SPACE STATS:
Total nodes generated          =  1276107  (includes start)
Nodes discarded by loop_check  =   776106  (500001 distinct states added to queue)
Nodes tested (by goal_test)    =   178141  (all expanded)
Nodes left in queue            =   321859

Time taken = 199.8782 seconds



'NODE_LIMIT_EXCEEDED'

In [ ]:
#loop check is false

In [ ]:
search( RW_PROBLEM_1, 'BF/FIFO', 5000, loop_check=False, heuristic=TasksRemaining, cost = cost)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=cost, heuristic=TasksRemaining
Max search nodes: 5000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

!! Search node limit (5000) reached !!
): No solution found :(


SEARCH SPACE STATS:
Total nodes generated          =     5001  (includes start)
Nodes tested (by goal_test)    =      917  (all expanded)
Nodes left in queue            =     4083

Time taken = 0.5187 seconds



'NODE_LIMIT_EXCEEDED'

In [ ]:
search( RW_PROBLEM_1, 'BF/FIFO', 50000, loop_check=False, heuristic=TasksRemaining, cost = cost)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=cost, heuristic=TasksRemaining
Max search nodes: 50000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
........
!! Search node limit (50000) reached !!
): No solution found :(


SEARCH SPACE STATS:
Total nodes generated          =    50001  (includes start)
Nodes tested (by goal_test)    =     8433  (all expanded)
Nodes left in queue            =    41567

Time taken = 6.458 seconds



'NODE_LIMIT_EXCEEDED'

In [ ]:
search( RW_PROBLEM_1, 'BF/FIFO', 500000, loop_check=False, heuristic=TasksRemaining, cost = cost)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=cost, heuristic=TasksRemaining
Max search nodes: 500000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
..............................................................................
!! Search node limit (500000) reached !!
): No solution found :(


SEARCH SPACE STATS:
Total nodes generated          =   500001  (includes start)
Nodes tested (by goal_test)    =    78357  (all expanded)
Nodes left in queue            =   421643

Time taken = 75.8145 seconds



'NODE_LIMIT_EXCEEDED'

In [ ]:
search( RW_PROBLEM_1, 'BF/FIFO', 5000000, loop_check=False, heuristic=TasksRemaining, cost = cost)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=cost, heuristic=TasksRemaining
Max search nodes: 5000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
.................................................................................................... (100000)
.................................................................................................... (200000)
.................................................................................................... (300000)
.................................................................................................... (400000)
.................................................................................................... (500000)
.................................................................................................... (600000)
.....................

'NODE_LIMIT_EXCEEDED'

In [ ]:
#A* case 1

In [ ]:
search( RW_PROBLEM_1, 'BF/FIFO', 5000, loop_check=False, heuristic=MoveMeds, cost = cost)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=cost, heuristic=MoveMeds
Max search nodes: 5000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

!! Search node limit (5000) reached !!
): No solution found :(


SEARCH SPACE STATS:
Total nodes generated          =     5001  (includes start)
Nodes tested (by goal_test)    =      920  (all expanded)
Nodes left in queue            =     4080

Time taken = 0.4986 seconds



'NODE_LIMIT_EXCEEDED'

In [ ]:
search( RW_PROBLEM_1, 'BF/FIFO', 50000, loop_check=False, heuristic=MoveMeds, cost = cost)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=cost, heuristic=MoveMeds
Max search nodes: 50000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
........
!! Search node limit (50000) reached !!
): No solution found :(


SEARCH SPACE STATS:
Total nodes generated          =    50001  (includes start)
Nodes tested (by goal_test)    =     8680  (all expanded)
Nodes left in queue            =    41320

Time taken = 5.2061 seconds



'NODE_LIMIT_EXCEEDED'

In [ ]:
search( RW_PROBLEM_1, 'BF/FIFO', 500000, loop_check=False, heuristic=MoveMeds, cost = cost)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=cost, heuristic=MoveMeds
Max search nodes: 500000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
...............................................................................
!! Search node limit (500000) reached !!
): No solution found :(


SEARCH SPACE STATS:
Total nodes generated          =   500001  (includes start)
Nodes tested (by goal_test)    =    79747  (all expanded)
Nodes left in queue            =   420253

Time taken = 97.6318 seconds



'NODE_LIMIT_EXCEEDED'

In [69]:
search( RW_PROBLEM_1, 'BF/FIFO', 5000000, loop_check=False, heuristic=MoveMeds, cost = cost)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=cost, heuristic=MoveMeds
Max search nodes: 5000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
.................................................................................................... (100000)
.................................................................................................... (200000)
.................................................................................................... (300000)
.................................

KeyboardInterrupt: 

In [70]:
search( RW_PROBLEM_1, 'BF/FIFO', 5000, loop_check=True, heuristic=MoveMeds, cost = cost)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=cost, heuristic=MoveMeds
Max search nodes: 5000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
.
!! Search node limit (5000) reached !!
): No solution found :(


SEARCH SPACE STATS:
Total nodes generated          =     8715  (includes start)
Nodes discarded by loop_check  =     3714  (5001 distinct states added to queue)
Nodes tested (by goal_test)    =     1372  (all expanded)
Nodes left in queue            =     3628

Time taken = 1.0738 seconds



'NODE_LIMIT_EXCEEDED'

In [71]:
search( RW_PROBLEM_1, 'BF/FIFO', 50000, loop_check=True, heuristic=MoveMeds, cost = cost)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=cost, heuristic=MoveMeds
Max search nodes: 50000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
...............
!! Search node limit (50000) reached !!
): No solution found :(


SEARCH SPACE STATS:
Total nodes generated          =   105638  (includes start)
Nodes discarded by loop_check  =    55637  (50001 distinct states added to queue)
Nodes tested (by goal_test)    =    15556  (all expanded)
Nodes left in queue            =    34444

Time taken = 12.9813 seconds



'NODE_LIMIT_EXCEEDED'

In [72]:
search( RW_PROBLEM_1, 'BF/FIFO', 500000, loop_check=True, heuristic=MoveMeds, cost = cost)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=cost, heuristic=MoveMeds
Max search nodes: 500000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
.................................................................................................... (100000)
...............................................................................
!! Search node limit (500000) reached !!
): No solution found :(


SEARCH SPACE STATS:
Total nodes generated          =  1275341  (includes start)
Nodes discarded by loop_check  =   775340  (500001 distinct states added to queue)
Nodes tested (by goal_test)    =   179140  (all expanded)
Nodes left in queue            =   320860

Time taken = 190.5865 seconds



'NODE_LIMIT_EXCEEDED'

In [73]:
search( RW_PROBLEM_1, 'BF/FIFO', 5000000, loop_check=True, heuristic=MoveMeds, cost = cost)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=cost, heuristic=MoveMeds
Max search nodes: 5000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
.................................................................................................... (100000)
..............................................................................................
:-)) *SUCCESS* ((-:

Path length = 14
Goal state is:
Robot location: Waste Disposal
Robot carrying: []
Room contents: {'Doctor Office': {'AOTD'}, 'Patient Ward': set(), 'Pharmacy': {'PresPackageC', 'PresPackageB', 'PresPackageA', 'PresPackageD'}, 'Waste Disposal': {'WasteE', 'WasteB', 'WasteC', 'WasteD', 'WasteA'}}
Cost of reaching goal: 14
The action path to the solution is:
    ('move to', 'Patient Ward')
    ('pick up', 'WasteB')
    ('pick up', 'WasteA')
    ('move to', 'Do

'GOAL_STATE_FOUND'

In [ ]:
#search( RW_PROBLEM_1, 'DF/LIFO', 500000, loop_check=False, randomise = False)

In [ ]:
#The robot must collect waste from every room and leave in in the Waste Disposal Room